# Discriminative use case: Sentiment Analysis

In this notebook, we will explore how we can perform classical ML task such as sentiment analysis using LLM.

### Use case description

Sentiment analysis is a common NLP task that involves classifying text into a pre-defined sentiment. For example, with Miyagi, we can classify a user's stock portfolio into a predefined categories based on current market sentiment, which can be determined using a Bing search.

> Note - To improve reliability of the model output, set the temperature to 0 and use top_k sampling. This will ensure that the model always generates the same output for the same input.

#### 1. Import Semantic Kernel dependencies

In [3]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230711.7-preview"
#r "nuget: Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230711.7-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.17.230711.7-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

Installed Packages Microsoft.SemanticKernel, 0.17.230711.7-preview Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230711.7-preview Microsoft.SemanticKernel.Skills.Web, 0.17.230711.7-preview

#### 2. Initialize kernel

In [4]:
var builder = new KernelBuilder();

var (_, model, azureEndpoint, apiKey, org, bingApiKey) = Settings.LoadFromFile();
builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);

IKernel kernel = builder.Build();

#### 3. Create a Semantic Function (Prompt Template)

In [1]:
string skPrompt = @"
This is a chat between a classification system and human looking to rate their stock portfolio. Given a portfolio in valid JSON, respond back a sentiment in JSON for the attribute: sentiment. To determine this attribute, consider Web results from Bing and propietary data below.

Example:
""stocks"": [
        {
            ""symbol"": ""MSFT"",
            ""allocation"": 0.3,
            ""sentiment"": ""positive""
        },
        {
            ""symbol"": ""ACN"",
            ""allocation"": 0.1,
            ""sentiment"": ""negative""
        },
        {
            ""symbol"": ""JPM"",
            ""allocation"": 0.3,
            ""sentiment"": ""neutral""
        },
        {
            ""symbol"": ""PEP"",
            ""allocation"": 0.3,
            ""sentiment"": ""neutral""
        }
    ]

Rate my portfolio:

{{$portfolio}}


Question: {{$input}}

Context: 
Web Results: {{$web}}
Propietary data: {{$propietary}}
";

// create semantic function
var textToSQL = kernel.CreateSemanticFunction(skPrompt, maxTokens: 1000, temperature: 0, topP: 0.5);
Console.WriteLine(textToSQL.ToString());

Error: (34,17): error CS0103: The name 'kernel' does not exist in the current context

#### 3. Create Context variables

In [4]:
var tableInfo = @"
    Users table has columns: user_id, first_name, last_name, and email.
    Transactions table has columns: transaction_id, user_id, transaction_amount, transaction_date, and transaction_type.
";

// context variables
var context = kernel.CreateNewContext();
context["dialect"] = "PostgreSQL";
context["tableInfo"] = tableInfo;

#### 4. Generate SQL from natural language

In [5]:
context["input"] = "Most expensive purchase in the last year";
var sql = await textToSQL.InvokeAsync(context);
Console.WriteLine(sql);


using var bingConnector = new BingConnector(Env.Var("BING_API_KEY"));
        var webSearchEngineSkill = new WebSearchEngineSkill(bingConnector);

        kernel.ImportSkill(webSearchEngineSkill, "WebSearch");
        kernel.ImportSkill(new LanguageCalculatorSkill(kernel), "advancedCalculator");
        // kernel.ImportSkill(new SimpleCalculatorSkill(kernel), "basicCalculator");
        kernel.ImportSkill(new TimeSkill(), "time");

        Console.WriteLine("*****************************************************");
        Stopwatch sw = new();
        Console.WriteLine("Question: " + question);

        var config = new Microsoft.SemanticKernel.Planning.Stepwise.StepwisePlannerConfig();
        config.ExcludedFunctions.Add("TranslateMathProblem");
        config.MinIterationTimeMs = 1500;
        config.MaxTokens = 4000;

        StepwisePlanner planner = new(kernel, config);
        sw.Start();
        var plan = planner.CreatePlan(question);

        var result = await plan.InvokeAsync(kernel.CreateNewContext());
        Console.WriteLine("Result: " + result);
        if (result.Variables.TryGetValue("stepCount", out string? stepCount))
        {
            Console.WriteLine("Steps Taken: " + stepCount);
        }

        if (result.Variables.TryGetValue("skillCount", out string? skillCount))
        {
            Console.WriteLine("Skills Used: " + skillCount);
        }

        Console.WriteLine("Time Taken: " + sw.Elapsed);
        Console.WriteLine("*****************************************************");
    }

Question: What was the most expensive purchase made in the last year?

SQLQuery: SELECT 
                t.transaction_id,
                t.transaction_date,
                t.transaction_amount,
                t.transaction_type,
                u.first_name,
                u.last_name
            FROM 
                transactions t
            JOIN 
                users u ON t.user_id = u.user_id
            WHERE 
                t.transaction_date >= DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 year')
                AND t.transaction_date < DATE_TRUNC('year', CURRENT_DATE)
            ORDER BY 
                t.transaction_amount DESC
            LIMIT 1;

Answer: The most expensive purchase made in the last year was {transaction_type} for {transaction_amount} by {first_name} {last_name} on {transaction_date}.
